In [19]:
import os
import azure.cognitiveservices.speech as speechsdk

## Speech recognition from microphone

Use the device's microphone to record audio and do speech recognition. Up to 30 seconds or silence is detected.

In [3]:
def recognize_from_microphone():
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
    speech_config.speech_recognition_language="my-MM"

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

In [9]:
recognize_from_microphone()

Speak into your microphone.
Recognized: အား လုံးပဲ မင်္ဂလာပါ တစ်ယောက် ပေး တဲ့ အတွက် ကျေးဇူးတင် ပါ တယ် နောက်မှ ပြန်တွေ့ကြမယ် ကျေးဇူးတင်ပါတယ်.


## Speech recognition from audio file

Below code use `recognize_once_async` so it only recognize up to 15 seconds of audio or until silence is detected.

In [14]:
def recognize_from_audio_async(fname):
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
    speech_config.speech_recognition_language="my-MM"

    audio_config = speechsdk.audio.AudioConfig(filename=fname)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Running speech recognition on audio file: {}".format(fname))
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

In [15]:
recognize_from_audio_async("audio_test_5min.wav")

Running speech recognition on audio file: audio_test_5min.wav
Recognized: အားလုံး မင်္ဂလာပါ will fight and အပတ်စဉ် ပုဂ္ဂိုလ် တင်ပြတာတွေ.


## Continuous recognition

In [20]:
# Create an instance of a speech config with specified subscription key and service region.
speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
speech_config.speech_recognition_language="my-MM"

# Create an audio configuration that points to an audio file.
audio_filename = "audio_test_5min.wav"
audio_input = speechsdk.AudioConfig(filename=audio_filename)

# Create a recognizer with the given settings
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

# Define callbacks for events
def recognized_cb(evt):
    print('Recognized: {}'.format(evt.result.text))

def canceled_cb(evt):
    print('CANCELED: Reason={}'.format(evt.reason))
    if evt.reason == speechsdk.CancellationReason.Error:
        print('CANCELED: ErrorDetails={}'.format(evt.error_details))

def session_stopped_cb(evt):
    print('Session stopped.')

# Connect callbacks to events
speech_recognizer.recognized.connect(recognized_cb)
speech_recognizer.canceled.connect(canceled_cb)
speech_recognizer.session_stopped.connect(session_stopped_cb)

In [21]:
# Start continuous recognition
# This will run continuously until the end of audio file or until you stop it.

speech_recognizer.start_continuous_recognition()

Recognized: အားလုံး မင်္ဂလာပါ will fight and အပတ်စဉ် ပုဂ္ဂိုလ် တင်ပြတာတွေ.
Recognized: ပြုလုပ်နေကြ ကြည့်တယ် လေ မျက်နှာ စကားဝိုင်း တစ်ချို့ ဆို ပါ တယ်.
Recognized: ဒီ လေးမျက်နှာ စကားဝိုင်း ကြောင် က တော့ တော် တော် များ လည်း သိ ပြီး ဖြစ် လောက် မယ် ထင် တယ် ကျွန်တော် ပြန် ပြီး မိတ်ဆက် ရင်တော့ ဒီ လေးမျက်နှာ စကားဝိုင်း က ဒီ မတူညီတဲ့ အမြင်တွေ အယူအဆ တွေ ရပ်တည်ချက် တွေ ဒီ အပေါ် မှာ ကျွန်တော်တို့ အမ်း ပြန် နားထောင် ပြီး ဆွေးနွေး ကြ တဲ့ သတ္တဝါ တွေ ဖြစ်တယ် ဒါ လုပ် ယဉ်ကျေးမှု အခြေခံ တဲ့ စကားဝိုင်း ဖြစ်တယ်.
Recognized: D.
Recognized: အမ်း.
Recognized: ကျွန်တော်တို့ မ တူညီ မှု တွေ ပေါ် ကနေ ပြီးတော့ ဘာတွေ ပို ကောင်း တဲ့ အဖြေ ရှာ နိုင် တယ် ပေါ့ နော် ဒီ အပေါ် မှာ ကျွန်တော် အဓိက ကြွယ် ပါ တယ် ဆို တော့ ဒီမှာ ဟို ကျွန်တော်တို့ speaker တွေ ကို အဓိက.
Recognized: ဗဟိုပြု တဲ့ ပုံစံ မျိုး မ ဟုတ် ဘူး ဒီ မှာ ၁ ယောက် တဲ့ လူ တွေ အကုန် လုံး က ဟို ကိုယ် ကိုယ် သိ ထား တဲ့ ရွာ ကို ဝင်ရောက် ဆွေးနွေးဖို့ ဖိတ်ခေါ်တယ်.
Recognized: အာ ဆွေးနွေး ချင်တယ် ဆိုရင် ကျွန်တော်တို့ လက်ထောင် ပြီး ဝင်ရောက် ဆွေးနွေး လို့ ရ တယ်.


In [22]:
# Run this to stop recognition
speech_recognizer.stop_continuous_recognition()

Recognized: ဆွေးနွေး တဲ့ အခါ မှာ တော့ အတတ်နိုင်ဆုံး တိုတို နဲ့ လို ရင် ဆွေးနွေး ပေး ဖို့ မေတ္တာရပ်ခံတယ် ဟို ပြော တဲ့ အခါ မှာ ဒီ ဒီ ကျွန်တော်တို့ ဆွေးနွေးမယ့် အကြောင်းအရာ နဲ့ သက်ဆိုင် တဲ့ ဟာ ကို ပဲ အဓိက ပြော.
Session stopped.
